<a href="https://colab.research.google.com/github/brunovianna/5d_cube/blob/main/comfyui_colab_krita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

# @markdown If you have space to store the models in google drive, the start time will be much shorter after the first installation

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
# USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = True
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

#@title Choice of models to download

DOWNLOAD_PARAMS = ""
# @markdown ---
# @markdown #### Choice of models to download
# @markdown (uses this script: https://github.com/Acly/krita-ai-diffusion/blob/main/scripts/download_models.py )

# @markdown The bare *minimum*, *recommended* or *all* models. *All* takes a lot of time to start and disk space
MINIMAL = False  #@param {type:"boolean"}
RECOMMENDED = True # @param {"type":"boolean","placeholder":"False"}
ALL = False  #@param {type:"boolean"}

# @markdown All needed to run SD1.5, SDXL, or Flux. Checkpoints not included.
SD15 = False  #@param {type:"boolean"}
SDXL = False  #@param {type:"boolean"}
FLUX = False  #@param {type:"boolean"}

# @markdown All checkpoints for selected workloads
CHECKPOINTS = False  #@param {type:"boolean"}

# @markdown Upscalers
UPSCALERS = False  #@param {type:"boolean"}

# @markdown Prefetch:  models which would be automatically downloaded on first use
PREFETCH = False  #@param {type:"boolean"}



DOWNLOAD_PARAMS += " --minimal" if MINIMAL else ""
DOWNLOAD_PARAMS += " --recommended" if RECOMMENDED else ""
DOWNLOAD_PARAMS += " --all" if ALL else ""
DOWNLOAD_PARAMS += " --sd15" if SD15 else ""
DOWNLOAD_PARAMS += " --sdxl" if SDXL else ""
DOWNLOAD_PARAMS += " --flux" if FLUX else ""
DOWNLOAD_PARAMS += " --checkpoints" if CHECKPOINTS else ""
DOWNLOAD_PARAMS += " --upscalers" if UPSCALERS else ""
DOWNLOAD_PARAMS += " --prefetch" if PREFETCH else ""


CHECKPOINT_LIST = ""

# @markdown Individual choice of models SD1.5
SERENITY = False  #@param {type:"boolean"}
DREAMSHAPER = False  #@param {type:"boolean"}
FLAT2DANIMERGE = False #@param {type:"boolean"}

# @markdown Individual choice of models SDXL
REALVIS = False  #@param {type:"boolean"}
ZAVYCHROMA = False  #@param {type:"boolean"}
PIXELWAVE = False #@param {type:"boolean"}

# @markdown Individual choice of models FLUX
DEV = False  #@param {type:"boolean"}
SCHNELL = False  #@param {type:"boolean"}


CHECKPOINT_LIST += " --checkpoint serenity" if SERENITY else ""
CHECKPOINT_LIST += " --checkpoint dreamshaper" if DREAMSHAPER else ""
CHECKPOINT_LIST += " --checkpoint flat2d-animerge" if FLAT2DANIMERGE else ""
CHECKPOINT_LIST += " --checkpoint realvis" if REALVIS else ""
CHECKPOINT_LIST += " --checkpoint zavychroma" if ZAVYCHROMA else ""
CHECKPOINT_LIST += " --checkpoint pixelwave" if PIXELWAVE else ""
CHECKPOINT_LIST += " --checkpoint flux-dev" if DEV else ""
CHECKPOINT_LIST += " --checkpoint flux-schnell" if SCHNELL else ""


print (DOWNLOAD_PARAMS)
print (CHECKPOINT_LIST)

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


 --recommended

Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.4 MB/s eta 0:00:00
/content/drive/MyDrive/ComfyUI/custom_nodes
/content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Already up to date.
/content/drive/MyDrive/ComfyUI
-= Install custom nodes dependencies =-

WARN: The `COMFYUI_PATH` environment variable is not set. Assuming 
`custom_nodes/ComfyUI-Manager/../../` as the ComfyUI path.
----------------------------------------------------------------------------------------------------
Restoring [1/6]: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
...
PyNaCl-1.5.0-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata
(8.6 kB)
............Downloading PyGithub-2.5.0-py3-none-any.whl

Download scripts and nodes

In [2]:
#
# model downloader node
!cd custom_nodes && git clone https://github.com/ciri/model-downloader-comfyui.git

# Krita models and nodes

# Controlnet Preprocessor nodes by Fannovel16
!cd custom_nodes && git clone https://github.com/Fannovel16/comfyui_controlnet_aux; cd comfyui_controlnet_aux && pip install -r requirements.txt

# ip-adapter
!cd custom_nodes && git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ;

# external tooling nodes
!cd custom_nodes && git clone https://github.com/Acly/comfyui-tooling-nodes.git

# inpaint
!cd custom_nodes && git clone https://github.com/Acly/comfyui-inpaint-nodes.git

# from https://github.com/Acly/krita-ai-diffusion/wiki/ComfyUI-Setup

!python -m pip install aiohttp tqdm websockets
#!cd /content/ComfyUI/krita-ai-diffusion/ && python scripts/download_models.py ../ --sd15 --sdxl --upscalers -- checkpoint serenity --checkpoint dreamshaper

if OPTIONS['USE_GOOGLE_DRIVE']:
    #change to content since it won't work in google drive
    !cd /content && git clone https://github.com/Acly/krita-ai-diffusion.git --recurse-submodules
    !cd /content/krita-ai-diffusion/ && python scripts/download_models.py /content/drive/MyDrive/ComfyUI $DOWNLOAD_PARAMS $CHECKPOINT_LIST
else:
    !git clone https://github.com/Acly/krita-ai-diffusion.git --recurse-submodules
    !cd krita-ai-diffusion/ && python scripts/download_models.py ../ $DOWNLOAD_PARAMS $CHECKPOINT_LIST





Cloning into 'model-downloader-comfyui'...
fatal: could not read Username for 'https://github.com': No such device or address
fatal: destination path 'comfyui_controlnet_aux' already exists and is not an empty directory.
fatal: destination path 'ComfyUI_IPAdapter_plus' already exists and is not an empty directory.
fatal: destination path 'comfyui-tooling-nodes' already exists and is not an empty directory.
fatal: destination path 'comfyui-inpaint-nodes' already exists and is not an empty directory.
Cloning into 'krita-ai-diffusion'...
remote: Enumerating objects: 7948, done.
remote: Counting objects: 100% (3564/3564), done.
remote: Compressing objects: 100% (663/663), done.
remote: Total 7948 (delta 3263), reused 2913 (delta 2901), pack-reused 4384 (from 1)
Receiving objects: 100% (7948/7948), 22.65 MiB | 45.29 MiB/s, done.
Resolving deltas: 100% (6256/6256), done.
Submodule 'ai_diffusion/websockets' (https://github.com/python-websockets/websockets.git) registered for path 'ai_diffusio

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2024-12-28 11:46:07--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.12.2/cloudflared-linux-amd64.deb [following]
--2024-12-28 11:46:07--  https://github.com/cloudflare/cloudflared/releases/download/2024.12.2/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/8f327838-88be-4fbe-822d-7ec8224c629c?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241228%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241228T114607Z&X-Amz-Expires=300&X-Amz-Signature=aa467b27cfc898eaa2bc69ab5eeddfae5f7e8919a4f0e2a8c0451a3933

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server